# Import packages

In [34]:
import pandas as pd
import numpy as np
import parse
import re
from bs4 import BeautifulSoup # handle html strings

# Read Dataset

In [73]:
filename = "../datasets/PropertyData_wDesc.csv"
# Default encoding (UTF-8) not working on this dataset
data = pd.read_csv(filename, encoding="ISO8859-1")
data.head()

,Postcode,Created,Advertised,Agreed,Completed,Date Listing Last Cancelled,Sale or Let,RTD3308_outside_space1 - Outside Space Description,EweMove Description S1 Features,EweMove Description S2 Description,...,Price / Rent,Price Qualifier,Sale Price % Achieved,Current EPC - EPC Expiry Date,DESC Council Tax Band,DESC Leasehold Ground Rent,DESC Leasehold Service Charge,ZPG_lease_expiry_years_remaining,# of Enquiry or viewings,# of Apps/Offers
0,LU7 4WN,03/04/2019,03/04/2019,NaN,NaN,28/02/2022,Sale,Communal Garden,<ul><li>CASH BUYERS ONLY</li><li>No Upper Chai...,Cash Buyers... You could pounce super quick on...,...,"<font color='blue'>&pound;140,000</font><br>Of...",Offers In Excess Of,130.71%,11/03/2028,Band B,NaN,NaN,68.0,32,12
1,L23 6YD,21/03/2019,24/03/2019,05/04/2021,28/06/2021,26/02/2020,Sale,NaN,<ul><li>Sea Views</li><li>Very Popular Locatio...,WATCHING THE SHIPS ROLL IN - from your very ow...,...,"<font color='blue'>&pound;325,000</font><br>Of...",Offers Over,NaN,28/03/2029,Band E,NaN,NaN,NaN,15,10
2,DA17 5PJ,25/03/2019,03/04/2019,25/01/2021,15/03/2021,13/06/2019,Sale,Rear Garden,<ul><li>Call NOW 24/7 or book instantly online...,"Guide price &pound;325,000 - &pound;350,000.Lo...",...,"<font color='blue'>&pound;325,000</font><br>Gu...",Guide Price,NaN,12/07/2028,Band D,NaN,NaN,NaN,14,4
3,NW4 1AP,28/06/2019,04/07/2019,NaN,NaN,NaN,Sale,NaN,<ul><li>5 bed link detached FREEHOLD house wit...,EweMove Hampstead - A rare opportunity to acqu...,...,"<font color='blue'>&pound;1,095,000</font><br>...",Guide Price,NaN,30/06/2029,NaN,NaN,NaN,NaN,13,1
4,GL7 5UX,12/06/2019,09/08/2019,13/04/2021,05/07/2021,NaN,Sale,Back Garden,<ul><li>Four Bedrooms</li><li>Detached</li><li...,For anyone looking for a family home in South ...,...,"<font color='blue'>&pound;395,000</font><br>",NaN,100.00%,30/06/2024,NaN,NaN,NaN,NaN,11,9


# Inspecting the dataset

In [36]:
data.shape

(3649, 35)

In [37]:
data.isnull().sum()

Postcode                                                 0
Created                                                  0
Advertised                                             453
Agreed                                                1381
Completed                                             1483
Date Listing Last Cancelled                           2179
Sale or Let                                              0
RTD3308_outside_space1 - Outside Space Description    1450
EweMove Description S1 Features                          0
EweMove Description S2 Description                     301
EweMove Description S3 Rooms                           977
EweMove Description S4 Summary                           0
RTD3307_parking1 - Parking Description                1331
RTD3307_parking2 - Parking Description                2077
RTD3307_parking3 - Parking Description                2779
RTD3308_outside_space2 - Outside Space Description    2001
RTD3308_outside_space3 - Outside Space Description    25

# Handling HTML text

## EweMove Description S1 Features

In [38]:
string = data["EweMove Description S1 Features"][0]
soup = BeautifulSoup(string, "html.parser")
string

'<ul><li>CASH BUYERS ONLY</li><li>No Upper Chain!</li><li>Very Rare 2 Car Parking Spaces</li><li>Great Commuter Links by Road & Rail</li><li>Communal Gardens</li><li>2 Bedrooms</li><li>Cul-De-Sac Location</li><li>Brand New Bathroom</li><li>Lovely Residential Area</li></ul>'

In [39]:
print(soup.prettify())

<ul>
 <li>
  CASH BUYERS ONLY
 </li>
 <li>
  No Upper Chain!
 </li>
 <li>
  Very Rare 2 Car Parking Spaces
 </li>
 <li>
  Great Commuter Links by Road &amp; Rail
 </li>
 <li>
  Communal Gardens
 </li>
 <li>
  2 Bedrooms
 </li>
 <li>
  Cul-De-Sac Location
 </li>
 <li>
  Brand New Bathroom
 </li>
 <li>
  Lovely Residential Area
 </li>
</ul>


In [40]:
features = soup.select("li")
for feature in features:
    print(feature.string.strip())

CASH BUYERS ONLY
No Upper Chain!
Very Rare 2 Car Parking Spaces
Great Commuter Links by Road & Rail
Communal Gardens
2 Bedrooms
Cul-De-Sac Location
Brand New Bathroom
Lovely Residential Area


## EweMove Description S3 Rooms

In [41]:
string = data["EweMove Description S3 Rooms"][6]
soup = BeautifulSoup(string, "html.parser")

In [42]:
rooms = soup.select("li")
for room in rooms:
    name = room.strong.string.split('-')[-1].strip()
    try:
        area = room.i.string
    except AttributeError:
        area = 1
    print("{:25s}{}".format(name, area))

Living Room              4.34m x 4.11m (17.8 sqm) - 14' 3" x 13' 5" (192 sqft)
Dining Room              3.13m x 2.83m (8.8 sqm) - 10' 3" x 9' 3" (95 sqft)
Kitchen                  3.12m x 2.83m (8.8 sqm) - 10' 2" x 9' 3" (95 sqft)
Play Room                2.29m x 2.04m (4.6 sqm) - 7' 6" x 6' 8" (50 sqft)
Bedroom 1                4.05m x 3.38m (13.7 sqm) - 13' 3" x 11' 1" (147 sqft)
Bedroom 2                4.11m x 2.29m (9.4 sqm) - 13' 5" x 7' 6" (101 sqft)
Bedroom 3                3.36m x 2.86m (9.6 sqm) - 11' x 9' 4" (103 sqft)
Garage                   5.09m x 2.48m (12.6 sqm) - 16' 8" x 8' 1" (136 sqft)


## EweMove Description S4 Summary

In [43]:
string = data["EweMove Description S4 Summary"][26]
soup = BeautifulSoup(string, "html.parser")

In [44]:
summary = list(filter(lambda s: s.startswith("<b>"), string.split("<li>")))
summary

['<b>HUGE Summerhouse</b><br>',
 '<b>Extra long garage</b><br>',
 '<b>Parking for 4 cars</b><br>',
 '<b>Great internal space</b><br>',
 "<b>South-facing garden</b></li><br><br>Call 24/7 to register your interest or go online to book a viewing at a time that suits you - it's that easy."]

In [45]:
single = parse.compile("<b>{}</b><br>")
double = parse.compile("<b>{}</b><br><br>{}<br><br>")
final1 = parse.compile("<b>{}</b><br><br>{}<br></li>")
final2 = parse.compile("<b>{}</b><br><br>{}<br></li><br><br>{}")
final3 = parse.compile("<b>{}</b></li>")
final4 = parse.compile("<b>{}</b></li><br><br>{}")
parsers = [single, double, final1, final2, final3, final4]
for s in summary:
    result = list(filter(lambda r: r is not None, map(lambda p: p.parse(s), parsers)))[0]
    print(result.fixed)

('HUGE Summerhouse',)
('Extra long garage',)
('Parking for 4 cars',)
('Great internal space',)
('South-facing garden', "Call 24/7 to register your interest or go online to book a viewing at a time that suits you - it's that easy.")


## Price / Rent

In [46]:
string = data["Price / Rent"][3632]
string

"<font color='blue'>&pound;0</font><br>"

In [47]:
price_qualifier = string.split("<br>")[-1]

pattern = "[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?"
price = re.findall(pattern, string)[0]

print(price_qualifier)
print(price)


0


# Generalize records

## Parking spaces

In [74]:
parkings_name = list(filter(lambda s: "parking" in s, data.columns))
parkings = data[parkings_name].copy()
parkings.head()

,RTD3307_parking1 - Parking Description,RTD3307_parking2 - Parking Description,RTD3307_parking3 - Parking Description
0,Allocated,Off Street,Residents
1,NaN,NaN,NaN
2,On Street,NaN,NaN
3,NaN,NaN,NaN
4,Driveway,Garage,Off Street


### Types of parking spaces for every property
First obtaining all the parking types in the original dataset, then get the parking types for every property

In [75]:
parkings = parkings.fillna("None")
types, counts = np.unique(parkings.to_numpy().flatten(), return_counts=True)
list(zip(types, counts))

[('Allocated', 343),
 ('Communal', 81),
 ('Covered', 36),
 ('Driveway', 1257),
 ('Garage', 842),
 ('Gated', 89),
 ('None', 6187),
 ('Off Street', 1209),
 ('On Street', 458),
 ('Permit', 70),
 ('Private', 258),
 ('Rear', 27),
 ('Residents', 90)]

In [76]:
def get_parking_spaces_for_property(first: str, second: str, third: str):
    parking_types = {"Allocated": 0, "Off Street": 0, "Residents": 0, "On Street": 0, "Driveway": 0, "Garage": 0,
                     "Permit": 0, "Private": 0, "Gated": 0, "Covered": 0, "Communal": 0, "Rear": 0}

    if first in parking_types.keys():
        parking_types[first] += 1
    if second in parking_types.keys():
        parking_types[second] += 1
    if third in parking_types.keys():
        parking_types[third] += 1

    return parking_types

In [77]:
parking_types = {key: [] for key in types if key != "None"}
for i in range(len(parkings)):
    first = parkings.iloc[i, 0]
    second = parkings.iloc[i, 1]
    third = parkings.iloc[i, 2]
    types = get_parking_spaces_for_property(first, second, third)

    list(map(lambda value, d: parking_types[d].append(types[value]), types.keys(), types.keys()))

In [78]:
for k, v in parking_types.items():
    print("{:10s}\t{:5d}\t{:5d}".format(k, sum(v), len(v)))

Allocated 	  343	 3649
Communal  	   81	 3649
Covered   	   36	 3649
Driveway  	 1257	 3649
Garage    	  842	 3649
Gated     	   89	 3649
Off Street	 1209	 3649
On Street 	  458	 3649
Permit    	   70	 3649
Private   	  258	 3649
Rear      	   27	 3649
Residents 	   90	 3649


### Number of parking spaces for every property

In [79]:
parkings = data[parkings_name]
parking_num = [parkings.iloc[i].count() for i in range(len(parkings))]
parking_num = pd.DataFrame({"parking num": parking_num})
parking_num.head()

,parking num
0,3
1,0
2,1
3,0
4,3


In [82]:
parking_num.sum()

parking num    4760
dtype: int64

## Outside spaces

In [101]:
outside_name = list(filter(lambda s: "outside_space" in s, data.columns))
outside = data[outside_name].copy()
outside.head()

,RTD3308_outside_space1 - Outside Space Description,RTD3308_outside_space2 - Outside Space Description,RTD3308_outside_space3 - Outside Space Description
0,Communal Garden,NaN,NaN
1,NaN,NaN,NaN
2,Rear Garden,Private Garden,NaN
3,NaN,NaN,NaN
4,Back Garden,Enclosed Garden,Private Garden


### Types of outside spaces for every property

In [55]:
def record_types(first: str, second: str, third: str, types: dict):
    if first in types.keys():
        types[first] += 1
    if second in types.keys():
        types[second] += 1
    if third in types.keys():
        types[third] += 1

    return types

In [90]:
outside = outside.fillna("None")
types, counts = np.unique(outside.to_numpy().flatten(), return_counts=True)
list(zip(types, counts))

[('Back Garden', 1366),
 ('Communal Garden', 227),
 ('Enclosed Garden', 735),
 ('Front Garden', 682),
 ('None', 6012),
 ('Patio', 431),
 ('Private Garden', 599),
 ('Rear Garden', 802),
 ('Terrace', 93)]

In [94]:
outside_types = {key: [] for key in types if key != "None"}
for i in range(len(outside)):
    types = {key: 0 for key in types if key != "None"}
    first = outside.iloc[i, 0]
    second = outside.iloc[i, 1]
    third = outside.iloc[i, 2]
    types = record_types(first, second, third, types)

    list(map(lambda value, d: outside_types[d].append(types[value]), types.keys(), types.keys()))

In [95]:
for k, v in outside_types.items():
    print("{:15s}{:5d}".format(k, sum(v)))

Back Garden     1366
Communal Garden  227
Enclosed Garden  735
Front Garden     682
Patio            431
Private Garden   599
Rear Garden      802
Terrace           93


### Number of outside spaces for every property

In [104]:
outside = data[outside_name]
outside_num = [outside.iloc[i].count() for i in range(len(outside))]
temp = pd.DataFrame({"outside_num": outside_num})
temp.head()

,outside_num
0,1
1,0
2,2
3,0
4,3


In [105]:
temp.sum()

outside_num    4935
dtype: int64